<a href="https://colab.research.google.com/github/heugyu/notebook/blob/master/SimilarGoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# 파일 업로드 기능 실행
uploaded = files.upload()

for fn in uploaded.keys():
    # 업로드된 파일 정보 출력
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving data.csv to data.csv
User uploaded file "data.csv" with length 14436764 bytes


In [43]:
import io
import pandas as pd 

# 업로드된 파일을 읽기 위해 io package의 StringIO 명령어 사용
df = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')))
df.head(20)

,flowpath_value,keyword_list,goods_list,reg_dt,svc_nation_cd
0,make cheese,make cheese,"496954259,524703350,670864957",2020-03-03,SG
1,anua,"anua,heartleaf 77% soothing toner",670841244,2020-03-03,SG
2,reduire,reduire,618840314,2020-03-03,SG
3,bodyluv cocomo,"bodyluv cocomo,washbasin tap,cocomo","645178237,653670643,672072113",2020-03-03,SG
4,betadine,betadine,"648605397,655860399",2020-03-03,SG
5,harpic,"harpic,toilet bowl cleaner,toilet,toilet bowl ...",659541927,2020-03-03,SG
6,toilet cleaner,"toilet cleaner,harpic toilet cleaner,harpic,to...","485605261,659541927",2020-03-03,SG
7,bluetooth earphone,"bluetooth earphone,awei earbuds,wireless earbu...","551560381,640626732,671740929,674643692",2020-03-03,SG
8,screen magnifier,screen magnifier,664066861,2020-03-03,SG
9,toothbrush holder,"toothbrush holder,toothpaste dispenser,toothbr...","580826618,612057931,656539862,665282636,665283...",2020-03-03,SG


In [0]:
import logging
from gensim.models import Word2Vec

### 기존 ML 유사 상품 학습 방법
- 동일 고객이 동일 키워드로 검색하고 액션이 있는 상품의 시퀀스 사용 
- 아이템 코드를 시퀀스하게 연결하여 하나의 문장으로 보고 각각의 상품을 하나의 단어로 생각 
- W2V 사용하여 임베딩하여 사전으로 저장 
- 아이템 코드를 임베딩된 사전으로 변환후 cos similar로 유사 판별
 

In [57]:
# 데이터 전처리
train = []
for idx in range(len(df)):
    row = df.iloc[idx]
    keyword_list = row.keyword_list.replace(' ', ',')
    goods_list = row.goods_list
    sentence = goods_list + ',' + keyword_list
    train.append(sentence.split(','))
train[:5]

[['496954259', '524703350', '670864957', 'make', 'cheese'],
 ['670841244', 'anua', 'heartleaf', '77%', 'soothing', 'toner'],
 ['618840314', 'reduire'],
 ['645178237',
  '653670643',
  '672072113',
  'bodyluv',
  'cocomo',
  'washbasin',
  'tap',
  'cocomo'],
 ['648605397', '655860399', 'betadine']]

In [0]:
# W2V 학습
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = Word2Vec(
    sentences=train,
    size=100,
    min_count=10,
    window=5,
    sg=0,
    iter=500,
    compute_loss=True
)


In [61]:
# 모델 검증 
model.wv.most_similar('dress')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('jacket', 0.4988751709461212),
 ('dresses', 0.4941546320915222),
 ('tops', 0.4205663204193115),
 ('blouse', 0.41724979877471924),
 ('girls', 0.40486112236976624),
 ('stockings', 0.3996584415435791),
 ('sweater', 0.3923976421356201),
 ('pants', 0.38954395055770874),
 ('blazer', 0.3842495083808899),
 ('safari', 0.37664562463760376)]